# Exercise: Perspective Projection

---

Prof. Dr.-Ing. Antje Muntzinger, Hochschule für Technik Stuttgart

antje.muntzinger@hft-stuttgart.de

---

In this exercise, we implement the perspective projection to transform a 3D world point to a 2d image point. We use a 3D cube as example object.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive, Layout
import ipywidgets as widgets

In [ ]:
def create_cube(size=2):
    """
    Create cube vertices and edges
    """
    # Define the vertices
    points = np.array([
        [-size, -size, -size],  # 0
        [size, -size, -size],   # 1
        [size, size, -size],    # 2
        [-size, size, -size],   # 3
        [-size, -size, size],   # 4
        [size, -size, size],    # 5
        [size, size, size],     # 6
        [-size, size, size]     # 7
    ])
    
    # Define edges as pairs of vertex indices
    edges = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # bottom face
        (4, 5), (5, 6), (6, 7), (7, 4),  # top face
        (0, 4), (1, 5), (2, 6), (3, 7)   # connecting edges
    ]
    
    return points, edges

def rotate_points(points, rx, ry, rz):
    """
    Rotate points around x, y, and z axes
    """
    # Convert angles to radians
    rx, ry, rz = np.radians([rx, ry, rz])
    
    # Rotation matrices
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    
    # Apply rotations
    R = Rz @ Ry @ Rx
    return points @ R.T

**TODO:** Implement the perspective project function below! The function should return a numpy array of size (8,2) with one projected point per row. You can check your code by running the cells below and inspecting the visualization.

In [ ]:
def perspective_project(points, focal_length):
    """
    Project 3D points onto 2D image plane using perspective projection
    """
    # YOUR CODE HERE
    
    return np.zeros((8, 2)) # replace with projected points

In [ ]:
def plot_3d_and_projection(points_3d, edges, focal_length, translation, rotation):
    """
    Create a figure with both 3D cube and its 2D projection
    """
    # Apply transformations
    tx, ty, tz = translation
    rx, ry, rz = rotation
    
    # First rotate
    transformed_points = rotate_points(points_3d, rx, ry, rz)
    # Then translate
    transformed_points = transformed_points + np.array([tx, ty, tz])
    
    # Create the projection
    points_2d = perspective_project(transformed_points, focal_length)
    
    # Create a figure with two subplots
    fig = plt.figure(figsize=(15, 7))
    
    # 3D plot
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Plot vertices
    ax1.scatter(transformed_points[:, 0], transformed_points[:, 1], transformed_points[:, 2], 
               c='blue', marker='o')
    
    # Plot edges
    for edge in edges:
        start = transformed_points[edge[0]]
        end = transformed_points[edge[1]]
        ax1.plot([start[0], end[0]], 
                [start[1], end[1]], 
                [start[2], end[2]], 'b-')
    
    # Plot camera center and image plane
    ax1.scatter([0], [0], [0], c='red', marker='^', s=100, label='Camera Center')
    
    # Create and plot image plane
    xx, yy = np.meshgrid([-3, 3], [-3, 3])
    zz = np.full_like(xx, focal_length)
    ax1.plot_surface(xx, yy, zz, alpha=0.2, color='gray')
    
    # Add viewing volume lines
    for x, y in [(-3, -3), (-3, 3), (3, -3), (3, 3)]:
        ax1.plot([0, x], [0, y], [0, focal_length], 'r--', alpha=0.3)
    
    # Set consistent axis limits
    limit = max(abs(transformed_points).max() * 1.2, 5)
    ax1.set_xlim([-limit, limit])
    ax1.set_ylim([-limit, limit])
    ax1.set_zlim([-limit, limit])
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('3D Cube and Camera Setup')
    
    # 2D projection plot
    ax2 = fig.add_subplot(122)
    
    # Plot projected vertices
    ax2.scatter(points_2d[:, 0], points_2d[:, 1], c='blue', marker='o')
    
    # Plot projected edges
    for edge in edges:
        start = points_2d[edge[0]]
        end = points_2d[edge[1]]
        ax2.plot([start[0], end[0]], 
                [start[1], end[1]], 'b-')
    
    # Set consistent axis limits for 2D plot
    limit_2d = max(abs(points_2d).max() * 1.2, 3)
    ax2.set_xlim([-limit_2d, limit_2d])
    ax2.set_ylim([-limit_2d, limit_2d])
    
    ax2.grid(True)
    ax2.set_aspect('equal')
    ax2.set_title('2D Projection')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    
    plt.tight_layout()
    plt.show()



In [ ]:
# Create initial cube
points_3d, edges = create_cube(size=2)

# Create interactive visualization
def update_visualization(focal_length, 
                        translate_x, translate_y, translate_z,
                        rotate_x, rotate_y, rotate_z):
    translation = [translate_x, translate_y, translate_z]
    rotation = [rotate_x, rotate_y, rotate_z]
    plot_3d_and_projection(points_3d, edges, focal_length, translation, rotation)

# Define slider styles
slider_style = {'description_width': '100px'}
slider_layout = Layout(width='400px')

# Create interactive controls
interactive_plot = interactive(
    update_visualization,
    focal_length=widgets.FloatSlider(
        value=1.0, min=0.5, max=3.0, step=0.1,
        description='Focal Length:',
        style=slider_style, layout=slider_layout
    ),
    translate_x=widgets.FloatSlider(
        value=0.0, min=-5.0, max=5.0, step=0.1,
        description='Translate X:',
        style=slider_style, layout=slider_layout
    ),
    translate_y=widgets.FloatSlider(
        value=0.0, min=-5.0, max=5.0, step=0.1,
        description='Translate Y:',
        style=slider_style, layout=slider_layout
    ),
    translate_z=widgets.FloatSlider(
        value=5.0, min=1.0, max=10.0, step=0.1,
        description='Translate Z:',
        style=slider_style, layout=slider_layout
    ),
    rotate_x=widgets.FloatSlider(
        value=0.0, min=-180.0, max=180.0, step=5.0,
        description='Rotate X (°):',
        style=slider_style, layout=slider_layout
    ),
    rotate_y=widgets.FloatSlider(
        value=0.0, min=-180.0, max=180.0, step=5.0,
        description='Rotate Y (°):',
        style=slider_style, layout=slider_layout
    ),
    rotate_z=widgets.FloatSlider(
        value=0.0, min=-180.0, max=180.0, step=5.0,
        description='Rotate Z (°):',
        style=slider_style, layout=slider_layout
    )
)

# Display the interactive visualization
display(interactive_plot)